# leave-site-out cross-validation

This notebook will implement a classifier on the ABIDE dataset that predicts autism from resting state fMRI data using leave-site-out cross-validation. This means that each fold contains every site but one, which is then used to measure performance on.

Let's first load the dataset so that we can extract the group information

In [1]:
from nilearn import datasets
import pandas as pd
#CHANGE PATH (data_dir) TO WERE YOU WANT TO STORE THE DATA
abide = datasets.fetch_abide_pcp(data_dir="/mnt/c/brainhackschool/project/data",
                                 pipeline="cpac",
                                 quality_checked=True)

# Transform phenotypic data into dataframe
abide_pheno = pd.DataFrame(abide.phenotypic)

# Extract group info
groups = []
for s in abide_pheno.SITE_ID:
    groups.append(s.decode()) # for some reason the site names are of type 'bytes'

/home/mikkel/miniconda3/lib/python3.7/site-packages/numpy/lib/npyio.py:2372: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


Next we define how to split our data set. Fortunately, there is a function in scikit-learn that splits data by groups, leaving one group in each iteration, which is exactly what we want here.

We will use the `prepare_data` function from the `prepare_data.py` we wrote to get X and y.

In [2]:
from sklearn.model_selection import LeaveOneGroupOut
import numpy as np
import prepare_data
import os

# define data and output directories
# if you already have downloaded the data, provide that directory
# THIS NEEDS TO BE CHANGED IN ORDER TO WORK FOR YOU
data_dir = os.path.join(os.sep, "mnt", "c", "brainhackschool", "project", "data")
output_dir = data_dir

X, y = prepare_data.prepare_data(data_dir, output_dir)

logo = LeaveOneGroupOut()
logo.get_n_splits(X, y, groups)

Loading dataset...


/home/mikkel/miniconda3/lib/python3.7/site-packages/numpy/lib/npyio.py:2372: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


Feature file found.
Running PCA...


20

Now we define our classifier. The classifier that is going to be used here is going to be a Linear Support Vector Classification (SVC).

In [3]:
from sklearn.svm import LinearSVC

l_svc = LinearSVC(max_iter=10000)

accuracy = []
counter = 0
for train_index, test_index in logo.split(X, y, groups):
    counter += 1
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("Training model", counter)
    l_svc.fit(X_train, y_train)
    acc = l_svc.score(X_test, y_test)
    accuracy.append(acc)

print(accuracy)

Training model 1
Training model 2
Training model 3
Training model 4
Training model 5
Training model 6
Training model 7
Training model 8
Training model 9
Training model 10
Training model 11
Training model 12
Training model 13
Training model 14
Training model 15
Training model 16
Training model 17
Training model 18
Training model 19
Training model 20
[0.5333333333333333, 0.6363636363636364, 0.6060606060606061, 0.6785714285714286, 0.6071428571428571, 0.5, 0.6046511627906976, 0.4, 0.6428571428571429, 0.74, 0.5769230769230769, 0.7407407407407407, 0.56, 0.5909090909090909, 0.5625, 0.6666666666666666, 0.686046511627907, 0.7941176470588235, 0.6716417910447762, 0.7073170731707317]
